In [1]:
import pandas as pd
import pybamm
import pybop

In [2]:
parameter_set = pybamm.ParameterValues("ECM_Example")
parameter_set.update(
    {
        "Cell capacity [A.h]": 5,
        "Nominal cell capacity [A.h]": 5,
        # "Current function [A]": 5,
        # "Initial SoC": 0.5,
        "Element-1 initial overpotential [V]": 0,
        "Upper voltage cut-off [V]": 4.2,
        "Lower voltage cut-off [V]": 3.0,
        "R0 [Ohm]": 1e-3,
        "R1 [Ohm]": 2e-4,
        "C1 [F]": 1e4,
        "Open-circuit voltage [V]": pybop.empirical.Thevenin().default_parameter_values[
            "Open-circuit voltage [V]"
        ],
    }
)
# Optional arguments - only needed for two RC pairs
parameter_set.update(
    {
        "R2 [Ohm]": 0.0003,
        "C2 [F]": 40000,
        "Element-2 initial overpotential [V]": 0,
    },
    check_already_exists=False,
)

In [3]:
model = pybop.empirical.Thevenin(
    parameter_set=parameter_set,
    options={"number of rc elements": 2},
    solver=pybamm.CasadiSolver(mode="safe", dt_max=10),
)

file_loc = r"../hppc_lut/G1/battery_G1_cycle_2_pulse_1_lut.csv"
df = pd.read_csv(file_loc, index_col=None, na_values=["NA"])
df = df.drop_duplicates(subset=["Time"], keep="first")

dataset = pybop.Dataset(
    {
        "Time [s]": df["Time"].to_numpy(),
        "Current function [A]": df["Current"].to_numpy(), 
        "Voltage [V]": df["Voltage"].to_numpy(),
    }
)

In [4]:
r_guess = 0.005
parameters = pybop.Parameters(
    pybop.Parameter(
        "R0 [Ohm]",
        prior=pybop.Gaussian(r_guess, r_guess / 10),
        bounds=[0, 0.2],
    ),
    pybop.Parameter(
        "R1 [Ohm]",
        prior=pybop.Gaussian(r_guess, r_guess / 10),
        bounds=[0, 0.2],
    ),
    pybop.Parameter(
        "R2 [Ohm]",
        prior=pybop.Gaussian(r_guess, r_guess / 10),
        bounds=[0, 0.2],
    ),
    pybop.Parameter(
        "C1 [F]",
        prior=pybop.Gaussian(500, 100),
        bounds=[1, 1500],
    ),
    pybop.Parameter(
        "C2 [F]",
        prior=pybop.Gaussian(2000, 500),
        bounds=[0, 25000],
    ),
)

In [10]:
initial_state_of_charge= df["SoC"].iloc[0] / 100 

model.build(
    # initial_state={"Initial open-circuit voltage [V]": df["Voltage"].to_numpy()[0]}
    initial_state={"Initial SoC": initial_state_of_charge} # soc scales between 0 to 1
)
problem = pybop.FittingProblem(
    model,
    parameters,
    dataset,
)

cost = pybop.SumSquaredError(problem)

In [11]:
optim = pybop.PSO(
    cost,
    sigma0=[1e-3, 1e-3, 1e-3, 50, 500],
    max_unchanged_iterations=30,
    max_iterations=100,
)
results = optim.run()

Halt: No significant change for 30 iterations.
OptimisationResult:
  Best result from 1 run(s).
  Initial parameters: [4.72230943e-03 5.26250582e-03 5.17024117e-03 4.18326468e+02
 1.98765223e+03]
  Optimised parameters: [3.39814064e-03 1.08408619e-02 2.23813877e-02 6.34041280e+02
 1.15871596e+00]
  Total-order sensitivities:
  Diagonal Fisher Information entries: None
  Final cost: 1.3994556385835333
  Optimisation time: 140.30535650253296 seconds
  Number of iterations: 76
  Number of evaluations: 441
  SciPy result available: No
  PyBaMM Solution available: Yes


In [12]:
pybop.plot.quick(problem, problem_inputs=results.x, title="Optimised Comparison");

In [14]:
pybop.plot.convergence(optim)
pybop.plot.parameters(optim);

: 

In [3]:
import os
import pandas as pd
import pybamm
import pybop

# 298.15 K // 25 degree C
FIXED_TEMPERATURE = 298.15

# Prepare an empty list to store LUT data for each pulse
lut = []

# Loop over pulses 0 to 9
for pulse in range(10):
    file_loc = f"../hppc_lut/G1/battery_G1_cycle_2_pulse_{pulse}_lut.csv"
    print(f"Processing {file_loc}...")
    
    # Read and clean the CSV data
    df = pd.read_csv(file_loc, na_values=["NA"])
    df = df.drop_duplicates(subset=["Time"], keep="first")
    
    # Create the dataset for pybop
    dataset = pybop.Dataset({
        "Time [s]": df["Time"].to_numpy(),
        "Current function [A]": df["Current"].to_numpy(), 
        "Voltage [V]": df["Voltage"].to_numpy(),
    })
    
    # Update parameter set for the ECM
    parameter_set = pybamm.ParameterValues("ECM_Example")
    parameter_set.update({
        "Cell capacity [A.h]": 5,
        "Nominal cell capacity [A.h]": 5,
        "Element-1 initial overpotential [V]": 0,
        "Upper voltage cut-off [V]": 4.2,
        "Lower voltage cut-off [V]": 3.0,
        "R0 [Ohm]": 1e-3,
        "R1 [Ohm]": 2e-4,
        "C1 [F]": 1e4,
        "Open-circuit voltage [V]": pybop.empirical.Thevenin().default_parameter_values["Open-circuit voltage [V]"]
    })
    parameter_set.update({
        "R2 [Ohm]": 0.0003,
        "C2 [F]": 40000,
        "Element-2 initial overpotential [V]": 0,
    }, check_already_exists=False)
    
    # Build the ECM model
    model = pybop.empirical.Thevenin(
        parameter_set=parameter_set,
        options={"number of rc elements": 2},
        solver=pybamm.CasadiSolver(mode="safe", dt_max=10),
    )
    
    # Extract the initial SoC from the CSV file (scaled from 0 to 1)
    initial_soc = df["SoC"].iloc[0] / 100
    model.build(initial_state={"Initial SoC": initial_soc})
    
    # Set up the parameters for the optimization
    r_guess = 0.005
    parameters = pybop.Parameters(
        pybop.Parameter("R0 [Ohm]", prior=pybop.Gaussian(r_guess, r_guess / 10), bounds=[0, 0.2]),
        pybop.Parameter("R1 [Ohm]", prior=pybop.Gaussian(r_guess, r_guess / 10), bounds=[0, 0.2]),
        pybop.Parameter("R2 [Ohm]", prior=pybop.Gaussian(r_guess, r_guess / 10), bounds=[0, 0.2]),
        pybop.Parameter("C1 [F]", prior=pybop.Gaussian(500, 100), bounds=[1, 1500]),
        pybop.Parameter("C2 [F]", prior=pybop.Gaussian(2000, 500), bounds=[0, 25000]),
    )
    
    # Define and solve the fitting problem
    problem = pybop.FittingProblem(model, parameters, dataset)
    cost = pybop.SumSquaredError(problem)
    optim = pybop.PSO(
        cost,
        sigma0=[1e-3, 1e-3, 1e-3, 50, 500],
        max_unchanged_iterations=30,
        max_iterations=100,
    )
    results = optim.run()
    
    # The results vector (results.x) -- contains the optimized parameters in order to "parameters":
    # R0, R1, R2, C1, C2. We are interested in R0, R1, and C1.
    r0, r1, r2, c1, c2 = results.x
    
    # Create a LUT entry for the current pulse
    pulse_entry = {
        #"pulse": pulse,
        "current": df["Current"][1],
        "voltage": df["Voltage"][0],
        "temperature": FIXED_TEMPERATURE,
        "r0": r0,
        "r1": r1,
        "c1": c1,
        "SoC": df["SoC"][0]
    }
    lut.append(pulse_entry)

# Optionally, convert the LUT list to a DataFrame and save as CSV
lut_df = pd.DataFrame(lut)
lut_df.to_csv("ecm_lut_table.csv", index=False)
print("LUT table saved to ecm_lut_table.csv")


Processing ../hppc_lut/G1/battery_G1_cycle_2_pulse_0_lut.csv...
Halt: No significant change for 30 iterations.
OptimisationResult:
  Best result from 1 run(s).
  Initial parameters: [5.28344161e-03 5.62985155e-03 6.63007884e-03 4.17312638e+02
 2.44960751e+03]
  Optimised parameters: [2.30722653e-02 1.58788269e-02 6.28197047e-03 2.79867601e+02
 2.41607978e+02]
  Total-order sensitivities:
  Diagonal Fisher Information entries: None
  Final cost: 0.6548965446717853
  Optimisation time: 108.31279754638672 seconds
  Number of iterations: 53
  Number of evaluations: 334
  SciPy result available: No
  PyBaMM Solution available: Yes
Processing ../hppc_lut/G1/battery_G1_cycle_2_pulse_1_lut.csv...
Halt: No significant change for 30 iterations.
OptimisationResult:
  Best result from 1 run(s).
  Initial parameters: [5.70979199e-03 5.77466189e-03 4.51342967e-03 4.88654928e+02
 1.97675036e+03]
  Optimised parameters: [2.63364755e-02 1.00724586e-02 1.16870247e-04 6.45544379e+02
 6.44014907e+03]
  To

In [10]:
import pandas as pd

# Define the input and output file paths
input_file = 'G1_cycle2_ecm_lut_table.csv'  # Replace with your input file path
output_dir = 'data/'  # Directory where the output files will be saved

# Read the input CSV file
data = pd.read_csv(input_file)

# Extract relevant columns (r0, r1, c1, and SoC)
r0_data = data[['temperature', 'current', 'SoC', 'r0']]
r1_data = data[['temperature', 'current', 'SoC', 'r1']]
c1_data = data[['temperature', 'current', 'SoC', 'c1']]

# Save each to a new CSV file
r0_data.to_csv(f'{output_dir}r0.csv', index=False)
r1_data.to_csv(f'{output_dir}r1.csv', index=False)
c1_data.to_csv(f'{output_dir}c1.csv', index=False)

print(f"Files have been saved to {output_dir}")

Files have been saved to data/
